# Explore JWST and HST images

In this notebook, we load Imviz and we use mainly the user interface to explore some JWST and HST images. The data can be downloaded at this link (images are quite large so they are not packaged into a single humongous zip file. Feel free to download as many as you like.

In [ ]:
import warnings
from jdaviz import Imviz
from astropy.coordinates import SkyCoord
from photutils.aperture import SkyCircularAperture
import astropy.units as u

Choose one of the two ways to run Imviz.

In [ ]:
# If in jupyter lab
imviz = Imviz()
imviz.show(loc='sidecar', height=1000)

In [ ]:
# If in notebook
imviz = Imviz()
imviz.show(height=700)

Here is a list of steps we can do in the UI:
- Import data. The HST file is ic8n06020_drz.fits. The others are all JWST files sorted by photometric band.
    - If you are on the platform, the data will be in /home/shared/preloaded-fits/jdaviz_data/UNCOVER.
- Link data by WCS
- Blink between data
- Deselect some dataset and keep only the JWST F150W and the HST file
- Blink between the two pressing b (wow to resolution)
- Zoom and pan and blink again
- Select only the JWST longest wavelength and the HST file
- Blink between the two (find objects we did not see with HST)
- Use the markers plugin to save the coordinates of your favorite objects
- Take a screenshot of your favorite object and post in slack
- Select a region around one of those objects in the JWST image
- Recenter appropriately
- Run aperture photometry and get an estimate of the flux
- Select only JWST images and create an RGB image
- Export (or screenshot) it and post in slack.

## Code to do the above steps programmatically

Change path to your local directory if you are not working on the platform.

In [ ]:
#data_dir = '/home/shared/preloaded-fits/jdaviz_data/UNCOVER/'
data_dir = './UNCOVER/'

files = ['jw02561-o001_t003_nircam_clear-f200w_i2d.fits',
         'jw02561-o001_t003_nircam_clear-f444w_i2d.fits',
         'ic8n06020_drz.fits'
        ]

labels = ['JWST F200W',
          'JWST F444W',
          'HST'
         ]

for i in range(0,len(files)):
    with warnings.catch_warnings():
        warnings.simplefilter('ignore')
        imviz.load_data(f'{data_dir}/{files[i]}', data_label=labels[i])

In [ ]:
viewer = imviz.default_viewer

In [ ]:
imviz.plugins

### Link by WCS

In [ ]:
linkwcs = imviz.plugins['Links Control']
linkwcs.link_type = 'WCS'

### Set better plot options

In [ ]:
plotopt = imviz.plugins['Plot Options']

plotopt.layer = 'HST[SCI,1]'
plotopt.stretch_function = 'Logarithmic'
plotopt.stretch_vmin = 0.52
plotopt.stretch_vmax = 30
plotopt.image_colormap = 'Gray'

plotopt.layer = 'JWST F200W[DATA]'
plotopt.stretch_function = 'Logarithmic'
plotopt.stretch_vmin = 0.12
plotopt.stretch_vmax = 20
plotopt.image_colormap = 'Viridis'

plotopt.layer = 'JWST F444W[DATA]'
plotopt.stretch_function = 'Logarithmic'
plotopt.stretch_vmin = 0.24
plotopt.stretch_vmax = 20
plotopt.image_colormap = 'Inferno'


Now to blink between the images you can press the 'b' key when the mouse is on the viewer. Land on the F444W filter for the next cell.

### Spot an object that is not in the HST data

In [ ]:
# Center the image on given sky coordinates.
sky = SkyCoord(ra=3.57984*u.degree, dec=-30.40156*u.degree, frame='icrs')
#sky = SkyCoord(ra=3.5757827*u.degree, dec=-30.4130878*u.degree, frame='icrs')
viewer.center_on(sky)

In [ ]:
# Set the zoom level directly.
viewer.zoom_level = 5

### Select a subset around the object

In [ ]:
interesting_object = SkyCircularAperture(sky, 0.5 * u.arcsec)
imviz.load_regions(interesting_object)

### Get aperture photometry for the object

In [ ]:
aperphot = imviz.plugins['Imviz Simple Aperture Photometry']
aperphot.open_in_tray()

This you have to do in the GUI because the API form the notebook is not ready yet. Make sure you select the correct layer that matches what you are seeing in the viewer.